In [1]:
import torch
from config import *

%reload_ext autoreload
%autoreload 2

In [2]:
ALPHA = CONFIG['ALPHA']

Load matrix $Q$

In [3]:
mat_q = torch.load(out_path('mat_q_resid.pt'))
mat_q = torch.abs(mat_q)
mat_q.shape

torch.Size([20352, 472])

Construct a correlation coefficient matrix
$$
R(i, j)=\frac{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))(z(s_j,t_k)-\tilde{z}(s_j))}{\sqrt{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))^2}\sqrt{\sum_{k=1}^d(z(s_j,t_k)-\tilde{z}(s_j))^2}},
$$
where $$\tilde{z}(s_i)=\frac{1}{d}\sum_{k=1}^dz(s_i,t_k)$$

In [4]:
d, p = mat_q.shape
mat_q_normalized = mat_q - mat_q.mean(dim=0)
mat_r = torch.zeros(p, p)
for i in range(p):
    for j in range(i, p):
        i_col = mat_q_normalized[:, i]
        j_col = mat_q_normalized[:, j]
        i_norm = max(torch.norm(i_col), 1e-12)
        j_norm = max(torch.norm(j_col), 1e-12)
        if i_norm == 0 or j_norm == 0:
            if i_norm == j_norm:
                mat_r[i][j] = 1
            else:
                mat_r[i][j] = 0
        else:
            mat_r[i][j] = (i_col @ j_col) / i_norm / j_norm
        mat_r[j][i] = mat_r[i][j]
mat_r = mat_r / mat_r.max()

torch.save(mat_r, out_path('mat_r.pt'))
mat_r.shape

torch.Size([472, 472])

Load matrix R

In [5]:
mat_r = torch.load(out_path('mat_r.pt'))
mat_r.shape

torch.Size([472, 472])

In [6]:
n_timesteps, n_sections = mat_q.shape
n_grouped = 0
groups = []

mat_r_copy = mat_r - torch.diag(mat_r.diag())
while n_grouped < n_sections:
    new_group_idx = torch.nonzero(mat_r_copy > ALPHA)
    if len(new_group_idx) > 0:
        corr = mat_r[new_group_idx[:, 0], new_group_idx[:, 1]]
        new_group_idx = new_group_idx[:, 0].unique()

        n_grouped += len(new_group_idx)
        mat_r_copy[new_group_idx, :] = 0
        mat_r_copy[:, new_group_idx] = 0
        groups.append((new_group_idx, corr.min(), corr.max()))
        if mat_r_copy.max() == 0:
            break
        else:
            mat_r_copy /= mat_r_copy.max()
del mat_r_copy

Analyse grouping

In [7]:
print(f'Using alpha={ALPHA}, {n_sections} correlated sections were divided ' +
      f'into {len(groups)} groups:')
n_ungrouped = 0
for i, (group, corr_min, corr_max) in enumerate(groups, start=1):
      group_coeffs = mat_r[tuple(group.T), :]
      print(f'Group {i} - {len(group)} sections with correlation coefficients '
          f'{corr_min:.3f} to {corr_max:.3f}')
print(str(n_sections - n_grouped) + " section(s) don't correlate with anything and weren't grouped")


Using alpha=0.9999, 472 correlated sections were divided into 159 groups:
Group 1 - 2 sections with correlation coefficients 1.000 to 1.000
Group 2 - 2 sections with correlation coefficients 0.981 to 0.981
Group 3 - 2 sections with correlation coefficients 0.976 to 0.976
Group 4 - 2 sections with correlation coefficients 0.952 to 0.952
Group 5 - 2 sections with correlation coefficients 0.288 to 0.288
Group 6 - 2 sections with correlation coefficients 0.258 to 0.258
Group 7 - 2 sections with correlation coefficients 0.247 to 0.247
Group 8 - 2 sections with correlation coefficients 0.225 to 0.225
Group 9 - 2 sections with correlation coefficients 0.220 to 0.220
Group 10 - 2 sections with correlation coefficients 0.213 to 0.213
Group 11 - 2 sections with correlation coefficients 0.191 to 0.191
Group 12 - 2 sections with correlation coefficients 0.187 to 0.187
Group 13 - 2 sections with correlation coefficients 0.183 to 0.183
Group 14 - 2 sections with correlation coefficients 0.182 to 0.1

/tmp/ipykernel_14303/3453684837.py:5: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  group_coeffs = mat_r[tuple(group.T), :]


In [8]:
representatives = torch.stack([g[0] for g, _, _ in groups])
mat_c = mat_q[:, representatives]
assert mat_c.shape == (mat_q.shape[0], len(groups))
torch.save(mat_c, out_path('mat_c.pt'))
mat_c.shape

torch.Size([20352, 159])

In [9]:
mat_x = torch.linalg.pinv(mat_c) @ mat_q
torch.save(mat_x, out_path('mat_x.pt'))
mat_x.shape

torch.Size([159, 472])